In [2]:
import basal_and_bark
from basal_and_bark import basal_and_bark as basal
from basal_and_bark import basal_and_bark_folium as basal_folium
import geemap
import ee
import ipyleaflet
from ipyleaflet import TileLayer

In [ ]:
m = geemap.Map()

In [ ]:
ee.Initialize()
# data = ee.Image('')
states = ee.FeatureCollection("TIGER/2018/States")

In [ ]:
st = states.filter(ee.Filter.eq('STUSPS', 'AL'))

In [4]:
leaflet_map = basal.Map(center = [40,-100], zoom = 4, test = "test", scroll_wheel_zoom = True)

In [5]:
df=leaflet_map.addGEEData('AL')
df
# .style \
#   .format(precision=3, thousands=".", decimal=",") \
#   .format_index(str.upper, axis=1)

{'max': {'treecover2000': 100},
 'mean': {'treecover2000': 62.0600837382026},
 'min': {'treecover2000': 0},
 'std': {'treecover2000': 43.85990269986808},
 'sum': {'treecover2000': 11165950540.913729}}

In [6]:
import pandas as pd

In [ ]:
label = {'max', 'mean', 'min', 'std', 'sum'}


In [7]:
#list(df.keys())[0]
lst = list(list(df.values()))#.values())

In [8]:
df = pd.DataFrame(lst)
df.index = ['max', 'mean', 'min', 'std', 'sum']
df.columns = ['GEEMap Data']
df

,GEEMap Data
max,1.000000e+02
mean,6.206008e+01
min,0.000000e+00
std,4.385990e+01
sum,1.116595e+10


In [ ]:
dem = ee.Image('UMD/hansen/global_forest_change_2018_v1_6')#ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL').mosaic()

vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

In [ ]:
dem_f = dem.clip(st)

In [ ]:
tile = leaflet_map.ee_tile_layer(dem_f.select('treecover2000'), vis_params)

In [ ]:
stats = geemap.image_stats(dem_f.select('treecover2000'), region =st, scale=30)
stats.getInfo()

In [ ]:
leaflet_map.add_layer(tile)

In [ ]:
leaflet_map

In [ ]:
def ee_tile_layer(
    ee_object, vis_params={}, name="Layer untitled", shown=True, opacity=1.0
):
    """Converts and Earth Engine layer to ipyleaflet TileLayer.
    Args:
        ee_object (Collection|Feature|Image|MapId): The object to add to the map.
        vis_params (dict, optional): The visualization parameters. Defaults to {}.
        name (str, optional): The name of the layer. Defaults to 'Layer untitled'.
        shown (bool, optional): A flag indicating whether the layer should be on by default. Defaults to True.
        opacity (float, optional): The layer's opacity represented as a number between 0 and 1. Defaults to 1.
    """

    image = None

    if (
        not isinstance(ee_object, ee.Image)
        and not isinstance(ee_object, ee.ImageCollection)
        and not isinstance(ee_object, ee.FeatureCollection)
        and not isinstance(ee_object, ee.Feature)
        and not isinstance(ee_object, ee.Geometry)
    ):
        err_str = "\n\nThe image argument in 'addLayer' function must be an instace of one of ee.Image, ee.Geometry, ee.Feature or ee.FeatureCollection."
        raise AttributeError(err_str)

    if (
        isinstance(ee_object, ee.geometry.Geometry)
        or isinstance(ee_object, ee.feature.Feature)
        or isinstance(ee_object, ee.featurecollection.FeatureCollection)
    ):
        features = ee.FeatureCollection(ee_object)

        width = 2

        if "width" in vis_params:
            width = vis_params["width"]

        color = "000000"

        if "color" in vis_params:
            color = vis_params["color"]

        image_fill = features.style(**{"fillColor": color}).updateMask(
            ee.Image.constant(0.5)
        )
        image_outline = features.style(
            **{"color": color, "fillColor": "00000000", "width": width}
        )

        image = image_fill.blend(image_outline)
    elif isinstance(ee_object, ee.image.Image):
        image = ee_object
    elif isinstance(ee_object, ee.imagecollection.ImageCollection):
        image = ee_object.mosaic()

    map_id_dict = ee.Image(image).getMapId(vis_params)
    tile_layer = TileLayer(
        url=map_id_dict["tile_fetcher"].url_format,
        attribution="Google Earth Engine",
        name=name,
        opacity=opacity,
        visible=shown,
    )
    return tile_layer